# Agents

The goal of this project is to allow a user to interact with a database with natural language. 

That is called DB RAG. 

The LLM does not know that a database exists, or the structure of the database. So for the user's inquirys about the database to have context, we must inform the LLM of that context. 

In LangChain, agents refer to a chat bot. 

Tools refer to functions available to agents. 

## LLMS' Tools

For any LLM, we can inform the agent that it has access to tools with a system message of the form 
```python
"""
You have access to the following tools
- run_query: runs a sqlite query and returns the result. Accepts an argument of a sql query as a string.
- ...

To use a tool always respond with the following format 
 {
    "name": <name of tool to use>
    "argument": <argument to pass to the tool>
 }
"""
```

However, it costs money to send this addition to the content over and over. 

OpenAI, who makes ChatGPT noticed that people were doing this frequently. They responded by extending ChatGPT to handle this exchange about tools in a more "natural" way.  


## ChatGPT Functions

ChatGPT functions, a newer more sophisticated concept, makes this more "programmer friendly". 

The user input is sent two lists; 
1. messages
    - giving the system, AI, and user messages as needed
2. functions
    - giving the list of each function available to the agent, including 
        - name of the function
        - a description that tells ChatGPT when it might be interested in using the function
        - the parameters that need arguments for the function (as the key `parameters`), and 
            - types for those arguments.
            - descriptions of the meaning of the argument


#### e.g. to GPT
The user sends a message to inform the LLM the question to answer/task to perfrom, as well as a list of tools/functions available. 

```JSON
messages=[
   {
      "role":"user",
      "content":"How many open orders are there?"
   }
   ]
functions = [
   {
      "name":"run_query",
      "description": "Run a sql query. Reutn the result.",
      "parameters": {
         "type":"object",
         "properties": {
            "query": {
               "type":"string",
               "description":"the sql query to execute",
            }
         }
      }
   }
]
```

This list is generated by instances of LangChain Tool objects. Those objects will refer to functions that we write and put in a folder called `tools`. 

#### e.g. From GPT
Chat GTP returns a JOSN object that indicates the LLM wants one of two things
1. to use a tool
   - wants to use one of the functions/tools ,
   - what property (that conform to the JSON schema sent to GPT) the LLM has chosen (e.g. line of code) 

```JSON
{"message": {
   "role": "assistant",
   "fucntion_call": {
      "name": "run_query",
      "arguments":{
         "query": "SELECT COUNT(*) FROM orders;" 
      }
   }
   }
}
```
2. Just wants to send a message to the user. Here, there will be no `"function_call"`, but rather content to be delivered to the user.
```JSON
{"message": {
   "role":"assistant",
   "content":"There are 1500 orders."
}}```


#### e.g. return to GPT

we then send a list of message back to GPT
- the initial message,
- the returned function call JSON 
- the results of running the SQL query GPT generated. 
- (list of functions too)

## tool from function

For our first example of building a ChatgGPT tool from a funciton, we will just run a SQL query. 

In [2]:
import sqlite3 

# Connect to the database. 
conn = sqlite3.connect("db.sqlite")

# Define a finction to be made into a tool. 
def run_sqlite_query(query):
    c =  conn.cursor() #Cursor is the object that allows us to access to the db.
    try:
        c.execute(query)
        return c.fetchall() 
    except sqlite3.OperationalError as err:
        return f"The following error occuerd: {str(err)}"

# Create tool 
from langchain.tools import Tool

run_query_tool = Tool.from_function(
    name="run_sqlite_query", #name of tool need not be name of function
    description="Run a sqlite query.",
    # Assign the function above as the function for this tool.
    func=run_sqlite_query,
    # args_schema=RunQueryArgsSchema # provides specific 
)

## Agent

**Definition:** An <u>agent</u> is a chain that can accept a list of tools and knows how to make use of them.

**Definition:** An <u>agent executor</u> is a while loop that runs an agent in interaction with the LLM iteratively unitl the response from the LLM is not a request to call a tool.

Note about the cell below The LLM makes assuptions about the structure of the database; we will prevent those assumptions later.

In [4]:
# from langchain.chat_models import ChatOpenAI # depricated
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.agents import OpenAIFunctionsAgent
from langchain.agents import AgentExecutor
from dotenv import load_dotenv

# The tool has been written into tools/sql.py. 
from tools.sql import run_query_tool 


load_dotenv() # for David Cherney's OpenAI account.
chat=ChatOpenAI()

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("{input}"),
        # AgentExecutor's loop requires simple memory: agent_scratchpad.
        # It takes in the input variable and then expands into
        # a new list of messages. 
        # The assignment "variable_name = 'agent_scratchpad' " is required.
        MessagesPlaceholder(variable_name = "agent_scratchpad")
        ]
    )

# List our tools. 
tools=[run_query_tool]

# Equivalent to the function `langchain.agents.initialize_agent`
agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = # no memory at this point. 
)

# A while loop that runs the agent iteratively 
# unitl the response is not a request to call a tool (a function call).
# In particular, untill the message does not have a "function_call" key
# but only a "content" key with value a message intended for the user.
# So, fancy name, not a fancy concept.
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

input="How many geese are in the database?"
agent_executor(input)



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `{'query': "SELECT COUNT(*) FROM birds WHERE species = 'Goose'"}`


The following error occuerd: no such table: birdsI'm sorry, but it seems that there is no table named "birds" in the database. Can you please provide more information about the database structure or the table where the geese information is stored?

> Finished chain.


{'input': 'How many geese are in the database?',
 'output': 'I\'m sorry, but it seems that there is no table named "birds" in the database. Can you please provide more information about the database structure or the table where the geese information is stored?'}

## Ignorance of database. 

The agent responded to

```text
"How many geese are in the database?"
```

with

```text
Invoking: `run_sqlite_query` with `{'query': 'SELECT COUNT(*) FROM geese'}`
```

The agent assumed there was a table named `geese` in the database. There is not.

We need to make our agent aware of the names of the tables. We do so using
1. a system message
2. an addtional tool

### 1. Add a system message

We can explicitly tell the LLM 
- to not make assumptions about the database (like it wants to).
- what tables are available. 

#### Obtain table names 
Our tool should be built from a function that takes in the list of all table names and returns the schema for each table.

In [5]:
# Modify prompt to inform ChaptGPT of the list of tables that exist.

# Obtain the list of tables from the database.
def list_tables():
    c = conn.cursor()
    # List tables in the database using SQL.
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    rows = c.fetchall() 
    # SQL format issue:
    # Each item in rows was a 2-tuple with first item a table name as a string, 
    # second item blank. Replace all toples with their fist entry.
    rows = [row[0] for row in rows if row[0] is not None]
    # Reformat as a string with the name of one table on each line. 
    tables = " ".join(rows)
    return tables

tables = list_tables()

# Some formatting is needed to get a list. 
print(f"The list of tables names is \n{tables.split(' ')}.")

The list of tables names is 
['users', 'addresses', 'products', 'carts', 'orders', 'order_products'].


#### Modify template

In [6]:
# Modify the prompt to include a system message about the list of tables.
from langchain.schema import SystemMessage

prompt = ChatPromptTemplate(
    messages=[
        # A custom hardcoded system message tells ChatGPT that 
        # there is a database, and what its tables are.
        SystemMessage(
            content = (
                "You are an AI that has access to a sqlite database. "
                f"The database has the following tables: {list_tables().split(' ')}\n"
                "Do not make any assumptions about what tables or columns exist. "
                "Instead, use the 'descibe_tables' function."
                )
            ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(
            variable_name = "agent_scratchpad"
            )
        ]
    )

### 2. Add a tool to describe tables

The following cell is an example of returning the schema of 3 tables just to familiarize you with the relevant SQL.

In [7]:
# c = conn.cursor()
# schemas_tuples = c.execute(
#     "SELECT sql "
#     "FROM sqlite_master " 
#     # f"WHERE type='table' and name IN ('users', 'addresses', 'products');"
#     f"WHERE type='table' and name IN {tuple(tables.split(' ')[:3])};"
#     )
# # # It seems like I get to use this iterative once and then it vanishes. 
# schemas_tuples = list(schemas_tuples) # Seems to make it more permananet.

# print(f"Schema tuples are of type {type(schema_tuple)} and length {len(schema_tuple)}.")


# for i, schema_tuple in enumerate(schemas_tuples):
#     print(f">>> The schema for table `{tables.split(' ')[:3][i]}` is")
#     print(schema_tuple[0])

#Again, the returned SQL is an iterable of 1-tuples. Thus, replace tuples with first components. 

# schemas = [schema_tuple[0] for schema_tuple in schemas_tuples 
#            if schema_tuple[0] is not None]

# schemas_string = '\n'.join(schemas)
# print(schemas_string)

#### Function

In [8]:
# Construct a new tool that describes the available tables. 
def describe_tables(table_names):
    c = conn.cursor()
    # Create a string that is a list of tables for SQL syntax. 
    # e.g. tables = "WHERE name in ('users','orders','products');" .
    tables = ', '.join("'"+ table + "'" for table in table_names) 
    schemas_tuples = c.execute(
        "SELECT sql "
        "FROM sqlite_master " 
        f"WHERE type='table' and name IN ({tables});"
        )
    schemas_tuples = list(schemas_tuples) # Seems to make it more permananet.
    schemas = [schema_tuple[0] for schema_tuple in schemas_tuples 
           if schema_tuple[0] is not None]
    schemas_string = '\n'.join(schemas)
    return schemas_string

# ts = tables.split(" ")

print(describe_tables(table_names = list_tables().split(' ')))

CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT UNIQUE,
    password TEXT
    )
CREATE TABLE addresses (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    address TEXT
    )
CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL
    )
CREATE TABLE carts (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    quantity INTEGER
    )
CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    created TEXT
    )
CREATE TABLE order_products (
    id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    amount INTEGER
    )


#### Tool

Now to create the tool for this function. 

In [9]:
describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description=(
        "Gven a list of table names, " 
        + "returns the schema of those tables."
        ),
    func=describe_tables,
 #   args_schema=DescribeTablesArgsSchema 
)

In [10]:
# Specify the agent and its executor. 
tools=[run_query_tool, describe_tables_tool]

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

# A while loop that runs the agent iteratively 
# unitl the response is not a requent to call a tool (a function call).
# So, fancy name, not a fancy concept.
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

input="How many geese are in the database?"
agent_executor(input)



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `{'query': 'SELECT COUNT(*) FROM geese;'}`


The following error occuerd: no such table: geeseI'm sorry, but there is no table named "geese" in the database.

> Finished chain.


{'input': 'How many geese are in the database?',
 'output': 'I\'m sorry, but there is no table named "geese" in the database.'}

ChatGPT still assumed that a table named `geese` appeared in the database. However, it caught itself; it took in the error message and gave natural language output stating that no such table existed. That is improvement! 

## Pydantic for typing 

Pydantic allows the use of classes to make type specifications. This can be used to force the LLM to give the appropriate type for input to functions. For example, we will force ChatGPT to give a string to the function `run_sqlite_query`.

In [47]:
# from pydantic.v1 import BaseModel
from pydantic import BaseModel
from typing import List

# Create a class to specify types for the parameters of run_query_tool.
# It is a child class of BaseModel... whatever that is. 
class RunQueryArgsSchema(BaseModel):
    # The following line demands that instances of this class be created with a
    # string value for their property `query`.
    query: str

run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a sqlite query.",
    func=run_sqlite_query,
    # Make run_query_tool required to take in a string via Pydantic.
    args_schema=RunQueryArgsSchema 
)

# Create a class to speciify types for parameters of describe_tables_tool.
class DescribeTablesArgsSchema(BaseModel):
    tables_names: List[str]

describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description="Gven a list of table names, returns the schema of those tables.",
    func=describe_tables,
    # Force the tool to take in a list of strings via pytdantic. 
    args_schema=DescribeTablesArgsSchema 
)

In [50]:
# Try it out

# Specify the agent and its executor. 
tools=[run_query_tool, describe_tables_tool]

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

# A while loop that runs the agent iteratively 
# unitl the response is not a requent to call a tool (a function call).
# So, fancy name, not a fancy concept.
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

input= "What is the address of the user ID that has the most items in its shoppping cart? "
agent_executor(input)



> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `{'tables_names': ['users', 'addresses', 'carts']}`


CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT UNIQUE,
    password TEXT
    )
CREATE TABLE addresses (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    address TEXT
    )
CREATE TABLE carts (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    quantity INTEGER
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT addresses.address FROM addresses INNER JOIN (SELECT carts.user_id, COUNT(carts.id) AS total_items FROM carts GROUP BY carts.user_id ORDER BY total_items DESC LIMIT 1) AS subquery ON addresses.user_id = subquery.user_id;'}`


[('53706 Baldwin Junctions, New Patricktown, OK 76166',), ('0712 Murphy Points, West Garrett, AS 94574',), ('0629 Vicki Court Apt. 451, Amandatown, CA 57682',), ('54399 Janice Springs, Kristenhaven, MI 84207',)]The address of the user with the most items in their s

{'input': 'What is the address of the user ID that has the most items in its shoppping cart? ',
 'output': 'The address of the user with the most items in their shopping cart is "53706 Baldwin Junctions, New Patricktown, OK 76166".'}

Enumerating tries;

1. Looks like it tried a few queries, got back errors in my custom format, tried again, several times, and then gave up  in the form of telling me what SQL code it thinks will work. That was not the idea. 


```sql
SELECT address
FROM addresses WHERE id = (
      SELECT address_id
        FROM users
          WHERE id = (
                SELECT user_id
                    FROM carts
                        WHERE id = (
                                  SELECT cart_id
                                        FROM order_products
                                              GROUP BY cart_id
                                                    ORDER BY COUNT(*) DESC
                                                          LIMIT 1     
                                    )
                        )
                            )
                                                            
```

2. It ran one query and was successful; 
```SQL
'SELECT addresses.address 
FROM addresses 
    JOIN carts ON addresses.user_id = carts.user_id 
GROUP BY addresses.user_id 
ORDER BY COUNT(carts.id) DESC 
LIMIT 1'
```
```JSON
[('53706 Baldwin Junctions, New Patricktown, OK 76166',)]
```
```text
> Finished chain.
{'input': 'What is the address of the user ID that has the most items in its shoppping cart? ',
 'output': 'The address of the user with the most items in their shopping cart is "53706 Baldwin Junctions, New Patricktown, OK 76166".'}
 ```

## Report in HTML

We will create a function that gives results in the form of HTML code for a table. 

Instances of Tool can only use a single argument. 
That is a legacy LangChain thing that will be changed someday...
For now, StructureTools are the multivariate version object.

We put the following in `tools/report.py`


In [54]:

from langchain.tools import StructuredTool 
# from pydantic.v1 import BaseSchema # Needs explanation. 
from pydantic import BaseModel

def write_report(filename, html):
    with open(filename, 'w') as f:
        f.write(html)

class WriteReportArgsSchema(BaseModel):
    filename: str
    html: str

write_report_tool = StructuredTool.from_function(
    name="write_report",
    description=("Write an HTML file to disk. "
    "Use this tool whenever someone asks for a report."),
    func=write_report,
    args_schema=WriteReportArgsSchema,
)

In [57]:
from tools.report import  write_report_tool

tools = [
    run_query_tool,
    describe_tables_tool,
    write_report_tool
]
# Re-instantiate the agent to have the new tools.
agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

# A while loop that runs the agent iteratively 
# unitl the response is not a requent to call a tool (a function call).
# So, fancy name, not a fancy concept.
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

input= """How many orders are there? Write a report.
"""
agent_executor(input)



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `{'query': 'SELECT COUNT(*) FROM orders'}`


[(1500,)]
Invoking: `write_report` with `{'filename': 'order_count_report.html', 'html': '<h1>Order Count Report</h1><p>There are 1500 orders in the database.</p>'}`


NoneI have generated the report. You can download it from [here](sandbox:/order_count_report.html).

> Finished chain.


{'input': 'How many orders are there? Write a report.\n',
 'output': 'I have generated the report. You can download it from [here](sandbox:/order_count_report.html).'}

There is no memory at this time; ChatGPT has no recollection of what it just did.

In [58]:
agent_executor("Do the same thing for users.")



> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `{'tables_names': ['users']}`


CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT UNIQUE,
    password TEXT
    )The 'users' table has the following columns:

1. id: INTEGER (Primary Key)
2. name: TEXT
3. email: TEXT (Unique)
4. password: TEXT

> Finished chain.


{'input': 'Do the same thing for users.',
 'output': "The 'users' table has the following columns:\n\n1. id: INTEGER (Primary Key)\n2. name: TEXT\n3. email: TEXT (Unique)\n4. password: TEXT"}

AgentScratchpad is not memory because it does not persist across user inputs; when one user input is given and there are multiple exchanges between AgentExecutor and ChatGPT (where the latter returns a function call each but the last time), the AgentScratchpad is a memory of those exchanges. As soon as the return from the LLM is not a function call, the scratchpad contents are deleted. 

How do we get memory into agents? 

Memory will only preserve a memory of the 
- first human message
- final AI message. 

and not the internediate_steps (exchanges that yield function calls).

In [60]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages =True
)

# Add MessagesPlaceholder to prompt template that holds memory.
prompt = ChatPromptTemplate(
    messages=[
        # A custom hardcoded system message tells ChatGPT that 
        # there is a database, and what its tables are.
        SystemMessage(
            content = (
                "You are an AI that has access to a sqlite database. "
                f"The database has the following tables: {tables}\n"
                "Do not make any assumptions "
                "about what tables or columns exist. "
                "Instead, use the 'descibe_tables' function."
                )
            ),
            #new:
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template(template="{input}"),
        MessagesPlaceholder(variable_name = "agent_scratchpad")
        ]
    )

# Add memory to the agent_executor. 
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools,
    memory=memory,
)

user_input1 = """Write a report titled 'The Most Ordered Thing' that describes 
the number of orders for the most ordered thing.
"""
agent_executor(user_input1)
user_input2 = 'Do the same for the second most ordered thing.'
agent_executor(user_input2)




> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `{'tables_names': ['orders', 'order_products', 'products']}`


CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL
    )
CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    created TEXT
    )
CREATE TABLE order_products (
    id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    amount INTEGER
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT product_id, COUNT(*) as order_count FROM order_products GROUP BY product_id ORDER BY order_count DESC LIMIT 1'}`


[(3929, 10)]
Invoking: `run_sqlite_query` with `{'query': 'SELECT name FROM products WHERE id = 3929'}`


[('Unbranded Pizza',)]
Invoking: `write_report` with `{'filename': 'The_Most_Ordered_Thing.html', 'html': "<h1>The Most Ordered Thing</h1><p>The most ordered thing is 'Unbranded Pizza' with 10 orders.</p>"}`


NoneI have written a report titled 'The Most Ordered Thing'. You can

{'input': 'Do the same for the second most ordered thing.',
 'chat_history': [HumanMessage(content="Write a report titled 'The Most Ordered Thing' that describes \nthe number of orders for the most ordered thing.\n", additional_kwargs={}, example=False),
  AIMessage(content="I have written a report titled 'The Most Ordered Thing'. You can download it from [here](sandbox:/The_Most_Ordered_Thing.html). According to the report, the most ordered thing is 'Unbranded Pizza' with 10 orders.", additional_kwargs={}, example=False),
  HumanMessage(content='Do the same for the second most ordered thing.', additional_kwargs={}, example=False),
  AIMessage(content='The second most ordered thing is the product with the ID 1437. Unfortunately, I don\'t have access to the specific details of this product, such as its name and price, as it is stored in the "products" table which I don\'t have the schema for.', additional_kwargs={}, example=False)],
 'output': 'The second most ordered thing is the produ

1. Wow... one time the second call to the agent executor gave this wild thing (an attempt to write and execute a python function to run a SQL query):
```text
OutputParserException: Could not parse tool input: 
{'name': 'python', 
'arguments': '
def get_second_most_ordered_product():\n    
    query = \'\'\'\n    
        SELECT p.product_name, 
        COUNT(op.order_id) as order_count\n    
        FROM products p\n    
        JOIN order_products op ON p.product_id = op.product_id\n    
        GROUP BY p.product_name\n    
        ORDER BY order_count DESC\n    
        LIMIT 1 OFFSET 1\n    
        \'\'\'\n    
        result = functions.run_sqlite_query({"query": query})\n    
        return result\n\n
        
second_most_ordered_product = get_second_most_ordered_product()\n
second_most_ordered_product'
} 
because the `arguments` is not valid JSON.
```


2. While it looked up the name for the first most ordered thing, when it got to the second call to the agent executor it was lazy and did not look up the name for the second most ordered thing;
```text
HumanMessage(content='Do the same for the second most ordered thing.', additional_kwargs={}, example=False),
  AIMessage(content='The second most ordered thing is the product with the ID 1437. Unfortunately, I don\'t have access to the specific details of this product, such as its name and price, as it is stored in the "products" table which I don\'t have the schema for.', additional_kwargs={}, example=False)],
```

### Conclusion

The behavior of the LLM has a very heavy random component. Prompt engineering needs to be applied by a human whenever there are bad results. For instance, I might need to say modify the description of `run_query_tool` to prevent any language other than SQL query language from being put in (since I got python one time... along with the assumption that there was a functions module containing the function `run_sqlite_query`).

```python
run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a SQLite query with input in SQL query language.",
    # Assign the function of this tool to the function above.
    func=run_sqlite_query,
    # args_schema=RunQueryArgsSchema # provides specific 
)
```

# Custom Handlers 

Debugging is considered difficult in LangChain.

Lets create a system to see the Sweet boxes around the messages exchanged using the package pyboxen in the file `handlers/chat_melde_start_handler.py`. 

In [ ]:
from pyboxen import boxen 

print(boxen("Text goes here", title="Huh",color="yellow"))

╭─ Huh ────────╮                                                                
│Text goes here│                                                                
╰──────────────╯                                                                



In [ ]:
# a helper function.
def boxen_print(*args,**kwargs):
    print(boxen(*args,**kwargs))
    
boxen_print(
    "ya and like I said text goes here", 
    title="asdf", 
    color="red"
    )

╭─ asdf ──────────────────────────╮                                             
│ya and like I said text goes here│                                             
╰─────────────────────────────────╯                                             



In [119]:
# pip install pyboxen

from langchain.callbacks.base import BaseCallbackHandler
from pyboxen import boxen


def boxen_print(*args, **kwargs):
    # Print to terminal in nice boxes.
    print(boxen(*args, **kwargs))



class ChatModelStartHandler(BaseCallbackHandler):
    # The name of the following method is mandatory to get 
    # messages when a chat message is started. 
    def on_chat_model_start(
            self, 
            serialized,  # Likely will never need
            messages, # Can be a list of list for batch jobs. Not here.
            **kwargs):
        print("\n\n\n\n========= Sending Messages =========\n\n")

        for message in messages[0]: # we have one list in the list; one batch.
            if message.type == "system":
                boxen_print(message.content, title=message.type, color="yellow")

            elif message.type == "human":
                boxen_print(message.content, title=message.type, color="green")

            elif (message.type == "ai" 
                  and 
                  "function_call" in message.additional_kwargs):
                call = message.additional_kwargs["function_call"]
                boxen_print(
                    f"Running tool {call['name']} " +
                        f"with args {call['arguments']}",
                    title=message.type,
                    color="cyan"
                )

            elif message.type == "ai":
                boxen_print(message.content, title=message.type, color="blue")

            elif message.type == "function":
                boxen_print(message.content, title=message.type, color="purple")

            else:
                boxen_print(message.content, title=message.type)

In [120]:
# Introduce the handler
from handlers.chat_model_start_handler import ChatModelStartHandler

handler = ChatModelStartHandler() 

# Put the handler in the list of callbacks for the Chat.

chat = ChatOpenAI(
    callbacks=[handler]
)

In [122]:
# re-instantiate the agent with the new chat instance.

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

agent_executor = AgentExecutor(
    agent=agent,
    # Turn off verbose so we can focus on the output of the handler.
    # verbose=True,
    tools=tools,
    memory=memory,
)
agent_executor("Write a report showing the most ordered item.")

agent_executor('Do the same for the second most ordered item.')





========= Sending Messages =========


╭─ system ─────────────────────────────────────────────────────────────────────╮
│You are an AI that has access to a sqlite database. The database has the      │
│following tables: users                                                       │
│addresses                                                                     │
│products                                                                      │
│carts                                                                         │
│orders                                                                        │
│order_products                                                                │
│Do not make any assumptions about what tables or columns exist. Instead, use  │
│the 'descibe_tables' function.                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

╭─ human ─────────────────────────────────────╮                  

KeyboardInterrupt: 